In [ ]:
import os,glob
import numpy as np
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf
import cv2
gpu=int(input("Which gpu number you would like to allocate:"))
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu)
model_name=int(input("Which model you would like to train(TYPE THE NUMBER ONLY LIKE 1,2,3)? 1. SE Inception v3    2. DENSENET 201   3. SE SQUEEZENET"))
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
import keras
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,Layer,ReLU, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from skimage.transform import radon, rescale
from skimage.filters import roberts, sobel, scharr, prewitt
from classification_models.keras import Classifiers
from skimage import feature
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import pickle
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D,LayerNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from tensorflow.keras.metrics import Recall, Precision
from skimage import data, exposure
from tensorflow.keras.layers import Layer
from PIL import Image
from numpy import asarray
from sklearn.utils import shuffle
import os
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf

def no_data_augmentation(normal_files,covid_files,pneumonia_files):
    aug_normal=[]
    aug_covid=[]
    aug_pneumonia=[]
    for ele in normal_files:
        #ele=ele/255.0
        x = Image.open(ele).convert('L')
        x = asarray(x)
        
        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_normal.append(pic)
    for ele in covid_files:
        #ele=ele/255.0
        x = Image.open(ele).convert('L')
        x = asarray(x)
        
        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_covid.append(pic)
    for ele in pneumonia_files:
        #ele=ele/255.0
        x = Image.open(ele).convert('L')
        x = asarray(x)
      
        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_pneumonia.append(pic)
    for i in range(len(aug_normal)):
        aug_normal[i]=aug_normal[i].reshape((224,224))
    
    for i in range(len(aug_covid)):
        aug_covid[i]=aug_covid[i].reshape((224,224))
    for i in range(len(aug_pneumonia)):
        aug_pneumonia[i]=aug_pneumonia[i].reshape((224,224))
    
    print("Normal files without augmentation:",len(aug_normal))
    print("Covid files without augmentation:", len(aug_covid))
    print("Pneumonia files without augmentation:",len(aug_pneumonia))
    return aug_normal,aug_covid,aug_pneumonia

def data_augmentation(normal_files,covid_files,pneumonia_files):
    aug_normal=[]
    aug_covid=[]
    thresh_hold=7
    aug_pneumonia=[]
    
    #x = tf.keras.preprocessing.image.load_img("/content/IM-0001-0001.jpeg")
    
    datagen=ImageDataGenerator(
        rotation_range=40,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,

    )
    #normal
    counter=0
    
    for location in tqdm(normal_files):
        counter=0

        x = Image.open(location).convert('L')
        x = asarray(x)
       
        x=cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        x=x/255.0
        x = np.expand_dims(x, axis=-1) 
        x=x.reshape((1,)+x.shape)
        #x=x/255.0


        for i in datagen.flow(x):
            if counter>=12:
                break
            #i=i/255.0

            #i = cv2.resize(i,(224,224),interpolation = cv2.INTER_CUBIC)
            aug_normal.append(i)
            counter+=1
    #covid
    counter=0
    for location in tqdm(covid_files):
        counter=0
        x = Image.open(location).convert('L')
        x = asarray(x)
    
        x=cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        x=x/255.0
        x = np.expand_dims(x, axis=-1) 
        #x=img_to_array(x)
        x=x.reshape((1,)+x.shape)
        #x=x/255.0


        for i in datagen.flow(x):
            if counter>=2:
                break

            aug_covid.append(i)
            counter+=1    
    #pneumonia
    counter=0
    for location in tqdm(pneumonia_files):
        counter=0
        x = Image.open(location).convert('L')
        x = asarray(x)
    

        x=cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        x=x/255.0
        x = np.expand_dims(x, axis=-1) 
        #x=img_to_array(x)
        x=x.reshape((1,)+x.shape)
        #x=x/255.0

        for i in datagen.flow(x):
            if counter>=3:
                break
            #i=i/255.0
            #i = cv2.resize(i,(224,224),interpolation = cv2.INTER_CUBIC)
            aug_pneumonia.append(i)
            counter+=1    

    for ele in normal_files:
        #ele=ele/255.0
        x = Image.open(ele).convert('L')
        x = asarray(x)
        
        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_normal.append(pic)
    for ele in covid_files:
        #ele=ele/255.0
        x = Image.open(ele).convert('L')
        x = asarray(x)
        
        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_covid.append(pic)
    for ele in pneumonia_files:
        #ele=ele/255.0
        x = Image.open(ele).convert('L')
        x = asarray(x)
      
        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_pneumonia.append(pic)
    for i in range(len(aug_normal)):
        aug_normal[i]=aug_normal[i].reshape((224,224))
    
    for i in range(len(aug_covid)):
        aug_covid[i]=aug_covid[i].reshape((224,224))
    for i in range(len(aug_pneumonia)):
        aug_pneumonia[i]=aug_pneumonia[i].reshape((224,224))
    
    print("Normal files after augmentation:",len(aug_normal))
    print("Covid files after augmentation:", len(aug_covid))
    print("Pneumonia files after augmentation:",len(aug_pneumonia))
    return aug_normal,aug_covid,aug_pneumonia

def making_full_data(aug_normal,aug_covid,aug_pneumonia):
    aug_normal=shuffle(aug_normal, random_state=0)
    aug_covid=shuffle(aug_covid,random_state=0)
    aug_pneumonia=shuffle(aug_pneumonia,random_state=0)
    
    aug_normal_labels=[]
    for i in range(len(aug_normal)):
        aug_normal_labels.append(0)
    print(np.shape(aug_normal),np.shape(aug_normal_labels))
    aug_covid_labels=[]
    for i in range(len(aug_covid)):
        aug_covid_labels.append(1)
    print(np.shape(aug_covid),np.shape(aug_covid_labels))
    aug_pneumonia_labels=[]
    for i in range(len(aug_pneumonia)):
        aug_pneumonia_labels.append(2)
    print(np.shape(aug_pneumonia),np.shape(aug_pneumonia_labels))  

    full_data=[]
    full_label=[]
    for i in range(len(aug_normal)):
        full_data.append(aug_normal[i])
        full_label.append(aug_normal_labels[i])
    for i in range(len(aug_covid)):
        full_data.append(aug_covid[i])
        full_label.append(aug_covid_labels[i])
    for i in range(len(aug_pneumonia)):
        full_data.append(aug_pneumonia[i])
        full_label.append(aug_pneumonia_labels[i])
        
    full_data=np.array(full_data)
    full_label=np.array(full_label)
    
    full_data=shuffle(full_data,random_state=0)
    full_label=shuffle(full_label,random_state=0)
    
    return full_data,full_label


    
def making_training_and_testing_data(full_data,full_label):
    
    
    train_label=[]
    for i in range(len(full_label)):
        if full_label[i]==0:
            train_label.append([0,1,0])
        elif full_label[i]==1:
            train_label.append([1,0,0])
        elif full_label[i]==2:

            train_label.append([0,0,1])
    full_label=np.array(train_label)
    
    return full_data,full_label

    



if __name__ == '__main__':  #straight away go to this
    normal_dir = "/home/sriparna/s_sharma/expert_system_review_results/Test_data_final/Normal" #give your normal cases data path here
    #vit_datasets/Dataset_ViT/ViT_dataset/Covid-19
    dir1 = os.path.join(normal_dir,"*.png")
    dir2 = os.path.join(normal_dir,"*.jpeg")
    dir = os.path.join(normal_dir,"*.jpg")
    normal_files = glob.glob(dir)
    normal_1 = glob.glob(dir1)
    normal_2 = glob.glob(dir2)
    normal_files.extend(normal_1)
    normal_files.extend(normal_2)

    normal_dir = "/home/sriparna/s_sharma/expert_system_review_results/Test_data_final/Covid"  #give your covid 19 cases data path here
    dir1 = os.path.join(normal_dir,"*.png")
    dir = os.path.join(normal_dir,"*.jpg")
    dir2 = os.path.join(normal_dir,"*.jpeg")
    covid_files = glob.glob(dir)
    covid_files2 = glob.glob(dir2)
    covid_files1 = glob.glob(dir1)
    covid_files.extend(covid_files2)
    covid_files.extend(covid_files1)

    normal_dir = "/home/sriparna/s_sharma/expert_system_review_results/Test_data_final/Pneumonia" #give your pneumonia cases data path here
    dir1 = os.path.join(normal_dir,"*.png")
    dir2 = os.path.join(normal_dir,"*.jpeg")
    dir = os.path.join(normal_dir,"*.jpg")
    pneumonia_files = glob.glob(dir)
    pneumonia_1 = glob.glob(dir1)
    pneumonia_2 = glob.glob(dir2)
    pneumonia_files.extend(pneumonia_1)
    pneumonia_files.extend(pneumonia_2)

    normal_files.sort()
    covid_files.sort()
    pneumonia_files.sort()
    normal_files=shuffle(normal_files,random_state=10)
    covid_files=shuffle(covid_files,random_state=10)
    pneumonia_files=shuffle(pneumonia_files,random_state=10)
    print("pneumonia_files:",len(pneumonia_files))
    print("covid_files:",len(covid_files))
    print("normal_files:",len(normal_files))
   
    test_normal_files=normal_files
    test_covid_files=covid_files
    test_pneumonia_files=pneumonia_files
    
    
    print("test normal:",len(test_normal_files))
    print("test covid:",len(test_covid_files))
    print("test pneumonia:",len(test_pneumonia_files))
 
    
    test_aug_normal,test_aug_covid,test_aug_pneumonia=no_data_augmentation(test_normal_files,test_covid_files,test_pneumonia_files)
    test_full_data,test_full_label=making_full_data(test_aug_normal,test_aug_covid,test_aug_pneumonia)
    test_full_data,test_full_label=making_training_and_testing_data(test_full_data,test_full_label)
    
    if model_name==1:  #IT WILL RUN SE Inception v3 MODEL  
        model1 = pickle.load(open('SE_Inception_v3_full_finetuning.sav', 'rb'))
        test_data=np.stack((test_full_data,)*3,axis=-1)
        model1.evaluate(test_data,test_full_label)
        
        model1 = pickle.load(open('SE_Inception_v3_partial finetuning.sav', 'rb'))
        test_data=np.stack((test_full_data,)*3,axis=-1)
        model1.evaluate(test_data,test_full_label)
        
        model1 = pickle.load(open('SE_Inception_v3_custom_finetuning.sav', 'rb'))
        test_data=np.stack((test_full_data,)*3,axis=-1)
        model1.evaluate(test_data,test_full_label)

    elif model_name==2: #IT WILL RUN DENSENET201 MODEL
      
        model1 = pickle.load(open('DenseNet201_full_finetuning.sav', 'rb'))
        test_data=test_full_data
        model1.evaluate(test_data,test_full_label)
        
        model1 = pickle.load(open('DenseNet201_partial_finetuning.sav', 'rb'))
        test_data=test_full_data
        model1.evaluate(test_data,test_full_label)
        
        model1 = pickle.load(open('DenseNet201_custom_finetuning.sav', 'rb'))
        test_data=np.stack((test_full_data,)*3,axis=-1)
        model1.evaluate(test_data,test_full_label)
    elif model_name==3: #IT WILL RUN SE SQUEEZENET MODEL
        model1 = pickle.load(open('se_squeeznet_full_finetuning.sav', 'rb'))
        test_data=test_full_data
        model1.evaluate(test_data,test_full_label)
        
        model1 = pickle.load(open('se_squeeznet_partial_finetuning.sav', 'rb'))
        test_data=test_full_data
        model1.evaluate(test_data,test_full_label)
        
        model1 = pickle.load(open('se_squeeznet_custom_finetuning.sav', 'rb'))
        test_data=test_full_data
        model1.evaluate(test_data,test_full_label)
    else:
        print("Check your input model - only 1,2,3  are allowed")


